In [58]:
FS = 128
SECONDS_PER_EPOCH = 10

OVERLAP = 0.5

In [59]:
import glob, mne
import numpy as np
import pandas as pd
import csv

with open('../data/DASPS.csv') as f:
    reader = csv.DictReader(f)

    d = {
        'subject_id': [],
        'sit_index': [],
        'valence': [],
        'arousal': []
    }

    for row in reader:
        _stripped = row['Id Participant'].strip('S')

        subject_id = int(_stripped) if _stripped else d["subject_id"][-1]
        sit_index = int(row['Id situation ']) - 1
        valence = int(row['valence'])
        arousal = int(row['Arousal'])
        
        d['subject_id'].append(subject_id)
        d['sit_index'].append(sit_index)
        d['valence'].append(valence)
        d['arousal'].append(arousal)

situations = pd.DataFrame(d)

print(situations)

def get_sam_label(valence, arousal):
    if valence > 5 or arousal < 5:
        return 0
    elif valence in [0, 1, 2] and arousal in [7, 8, 9]:
        return 3
    elif valence in [2, 3, 4] and arousal in [6]:
        return 2
    elif valence in [4, 5] and arousal in [5]:
        return 1
    
    return 0

def segment_file(path):
    orig_epochs = mne.read_epochs(path, preload=True)

    data = orig_epochs.get_data()
    data = np.concatenate(data, axis=1)

    print(data.shape)

    raw = mne.io.RawArray(data, orig_epochs.info)

    # Overlap is disabled!
    epochs = mne.make_fixed_length_epochs(raw, duration=SECONDS_PER_EPOCH, overlap=OVERLAP * SECONDS_PER_EPOCH * 0, preload=True)

    epochs.metadata = pd.DataFrame({k: [v[0]] * len(epochs) for k, v in orig_epochs.metadata.items()})

    dataset = orig_epochs.metadata["dataset"][0]
    subject_id = orig_epochs.metadata['subject'][0]

    # Add SAM labels to metadata
    if dataset == "dasps":
        samples_per_situation = FS * 30
        samples_per_epoch = FS * SECONDS_PER_EPOCH

        assert 30 % SECONDS_PER_EPOCH == 0, "Cannot segment into epochs that would match the situation length, please change SECONDS_PER_EPOCH to a divisor of 30"

        labels = []

        for i in range(len(epochs)):
            epochs_starts_at_sample = samples_per_epoch * i
            situation_idx = epochs_starts_at_sample // samples_per_situation

            # print(situation_idx)

            row = situations[(situations.subject_id == subject_id) & (situations.sit_index == situation_idx)]
            assert len(row) == 1

            labels.append(get_sam_label(row['valence'].values[0], row['arousal'].values[0]))

        epochs.metadata = epochs.metadata.assign(SAM=labels)

    epochs.save(path.replace("-epo.fif", f"_seg_{SECONDS_PER_EPOCH or 0}s-epo.fif"), overwrite=True)


paths = glob.glob("../data/fif/S??-epo.fif")

for path in paths:
    segment_file(path)

     subject_id  sit_index  valence  arousal
0             1          0        1        8
1             1          1        3        8
2             1          2        1        8
3             1          3        3        5
4             1          4        3        6
..          ...        ...      ...      ...
133          23          1        5        7
134          23          2        1        8
135          23          3        1        9
136          23          4        5        1
137          23          5        5        3

[138 rows x 4 columns]
Reading /Users/jakubblaha/Documents/thesis/thesis-python/notebooks/../data/fif/S10-epo.fif ...
    Found the data of interest:
        t =       0.00 ...   14992.19 ms
        0 CTF compensation matrices available
Adding metadata with 3 columns
12 matching events found
No baseline correction applied
0 projection items activated
(14, 23040)
Creating RawArray with float64 data, n_channels=14, n_times=23040
    Range : 0 ... 23039 =   

In [60]:
# Count SAM labels
paths = glob.glob(f"../data/fif/*_seg_{SECONDS_PER_EPOCH}s-epo.fif")

d = {'0': 0, '1': 0, '2': 0, '3': 0}

for path in paths:
    epochs = mne.read_epochs(path)

    for label in epochs.metadata['SAM']:
        d[str(label)] += 1

print(d)

Reading /Users/jakubblaha/Documents/thesis/thesis-python/notebooks/../data/fif/S12_seg_10s-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    9992.19 ms
        0 CTF compensation matrices available
Adding metadata with 4 columns
18 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/jakubblaha/Documents/thesis/thesis-python/notebooks/../data/fif/S20_seg_10s-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    9992.19 ms
        0 CTF compensation matrices available
Adding metadata with 4 columns
18 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/jakubblaha/Documents/thesis/thesis-python/notebooks/../data/fif/S04_seg_10s-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    9992.19 ms
        0 CTF compensation matrices available
Adding metadata with 4 columns
18 matching events found
No baseline correction applied
0 projection it